### 不使用框架，手工构建RAG，基于DeepSeek和Ollama生成模型

加载环境变量

In [12]:
from dotenv import load_dotenv
import os

load_dotenv()
# 配置 DeepSeek API 密钥
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
if not DEEPSEEK_API_KEY:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

1. 加载与读取文档

In [13]:
text = '';
with open('data/黑神话wiki.txt', 'rb') as f:
    text = f.read().decode('utf-8')

2. 分割文档
- 为了更精确地实现基于向量的语义检索，我们不能简单地嵌入文档的内容，而是需要分割：将文档分割成最合适的大小的多个知识块（Chunk）后做向量化

In [14]:
import re

def split_text_by_sentences(source_text: str, sentences_per_chunk: int = 3, overlap: int = 1) ->list[str]:
    """
    简单地把文档分割成多个知识块，每个知识块都包含指定数量的句子

    参数:
    source_text (str): 要分割的文本
    sentences_per_chunk (int): 每个知识块包含的句子数量，默认为3
    overlap (int): 相邻知识块之间的句子重叠数量，默认为1
    
    返回:
        list[str]: 分割后的知识块列表
    """
    # 参数检查
    if not isinstance(source_text, str):
        raise ValueError("输入必须是字符串类型")
    if not source_text.strip():
        raise ValueError("输入文本不能为空")
    if sentences_per_chunk < 1:
        raise ValueError("每个知识块至少包含1个句子")
    if overlap < 0 or overlap >= sentences_per_chunk:
        raise ValueError("重叠句子数必须大于等于0且小于每个知识块的句子数")
    
    # print(f"文本长度: {len(source_text)}")
    # print(f"文本内容前100个字符: {source_text[:100]}")
    
    # 简单化处理， 用正则表达式来分割句子
    sentences = re.split(r'(?<=[。！？.!?])\s*', source_text)
    # print(f"分割后的句子数量: {len(sentences)}")
    sentences = [s.strip() for s in sentences if s.strip()]

    # 如果句子数量不足，直接返回整个文本
    if len(sentences) <= sentences_per_chunk:
        return [source_text]
    
    chunks = []
    step = sentences_per_chunk - overlap
    
    # 生成知识块
    for i in range(0, len(sentences), step):
        chunk = sentences[i:i + sentences_per_chunk]
        if len(chunk) < sentences_per_chunk and i > 0:
            # 如果最后一个知识块句子不足，且不是第一个知识块，则跳过
            continue
        chunks.append(' '.join(chunk))
    
    return chunks

3. 设置嵌入模型

In [15]:
from sentence_transformers import SentenceTransformer

# 使用支持多语言的模型
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
chunks = split_text_by_sentences(source_text=text, sentences_per_chunk=8, overlap=2)
embeddings = embedding_model.encode(chunks)
print(f"文档向量维度: {embeddings.shape}")

文档向量维度: (21, 384)


4. 创建向量存储

In [16]:
# 创建向量存储
import chromadb
from chromadb.config import Settings

# 初始化 Chroma 客户端，设置持久化存储
client = chromadb.PersistentClient(path="./chroma_db")

# 创建或获取集合
collection = client.get_or_create_collection(
    name="black_myth_wiki",  # 集合名称
    metadata={"hnsw:space": "cosine"}  # 使用余弦相似度
)

# 准备数据
ids = [f"doc_{i}" for i in range(len(chunks))]
metadatas = [{"source": "黑神话wiki"} for _ in range(len(chunks))]

# 将数据添加到集合中
collection.add(
    embeddings=embeddings.tolist(),
    documents=chunks,
    ids=ids,
    metadatas=metadatas
)

print(f"成功存储了 {len(chunks)} 个文档块到向量数据库")

成功存储了 21 个文档块到向量数据库


4. 执行相似度检索

In [17]:
question = "黑神话悟空的游戏有什么特点?"

# 将问题转换为向量
question_embedding = embedding_model.encode(question)
# 在向量数据库中搜索最相似的文档
results = collection.query(
    query_embeddings=[question_embedding.tolist()],
    n_results=3  # 返回最相似的3个文档
)
# 打印检索结果
print("\n检索到的最相关文档片段：")
for i, (doc, score) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"\n片段 {i+1} (相似度得分: {1-score:.4f}):")
    print(doc)


检索到的最相关文档片段：

片段 1 (相似度得分: 0.1646):
[50][51]据电子游戏分析师丹尼尔·艾哈迈德（Daniel Ahmad）称，公布此段实机演示的主要目的是开发团队缺人，想通过这段视频吸引人才. [52]最初，开发团队只有7个人，但在游戏预告时，团队人数已增至约30人. [53]

2021年2月8日，开发团队释出了另一支3分钟的演示预告，以庆祝即将到来的农历牛年. 2021年8月20日开发团队再次释出一支12分钟的演示预告，由Epic Games研发的虚幻引擎5实机录制. [54]2022年8月20日，开发团队发布一支6分20秒剧情短片，[55]同时由GeForce再次发布一支8分22秒实机演示. [56]

制作人冯骥表示，《黑神话：悟空》是“黑神话系列”的第一部作品，内部的开发代号是“B1”，全称是“Black Myth One”. 根据他的构想，黑神话系列将是一个讲述中国东方神话体系里，不同传奇英雄的魔幻历险故事系列. 在规划这个系列的第一部作品之后，他对后两部黑神话游戏的名称和方向有一些想法，但尚未公开其内容.

片段 2 (相似度得分: 0.1490):
[47]

最后，天命人与代表孙悟空“执念”的大圣残躯战斗. 大圣残躯被击败后逐渐消散，紧箍儿掉入水中. 在决战之前如果天命人没有从二郎神那里获得孙悟空的第六件根器，孙悟空将不会复活，老猴子会将紧箍儿戴在天命人头上，天命人沦为天庭执行意志的新工具；但如果在决战之前天命人已经获得孙悟空的第六件根器，则六根齐聚，孙悟空成功复活，天命人成为孙悟空，老猴子虽然拿起了紧箍儿，但却没有给孙悟空戴上，孙悟空因此避免了天庭的束缚. [47][48]

游戏开发
《黑神话：悟空》由游戏科学于2017年开始开发，于2020年8月20日公布首段实机演示视频. [49]游戏预告片展示基于虚幻引擎4开发的游戏内容，主角在黑风山探索环境，并与各种敌人战斗. 发布一天内，预告片在YouTube上获得近200万次观看，在bilibili上则达到1000万次观看. [50][51]据电子游戏分析师丹尼尔·艾哈迈德（Daniel Ahmad）称，公布此段实机演示的主要目的是开发团队缺人，想通过这段视频吸引人才. [52]最初，开发团队只有7个人，但在游戏预告时，团队人数已增至约30人.

片段 3 (相似度得分:

5. 构建提示词

In [18]:
prompt = f"""
    请基于以下的上下文回答问题，如果上下文中不包含足够的回答问题的信息，请回答'我暂时无法回答该问题题'，不要编造。
    上下文：
    {doc}
    我的问题是：{question}
"""

6. 使用DeepSeek生成答案

In [19]:
from openai import OpenAI
client = OpenAI(
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com/v1"
)

response = client.chat.completions.create(
    model="deepseek-chat",  
    messages=[{
        "role": "user",
        "content": prompt
    }],
    max_tokens=1024
)
print(f"\n生成的答案: {response.choices[0].message.content}")


生成的答案: 根据上下文，《黑神话：悟空》的特点包括：  
1. **高销量与热度**：上线两周内全球销量超1800万份，销售额约61.65亿元，成为2024年8月北美、欧洲及日本PS5下载量最高的游戏。  
2. **实体版供不应求**：豪华版与收藏版需求远超供应，例如京东70万人抢购3万套实体版本。  
3. **内容争议**：部分平台（如WeGame）存在审查修改，例如将“酒”改为“甘露”，引发对游戏分级制度的讨论。  

上下文未提及其他具体玩法或设计特点（如战斗系统、画面等），故无法进一步回答。


7. 使用Ollama本地大模型生成答案

In [20]:
from ollama import chat

response = chat(
    model='qwen2.5:3b',  
    messages=[{
        "role": "user",
        "content": prompt
    }],
)
print(f"\n生成的答案: {response.message.content}")


生成的答案: 《黑神话：悟空》这款游戏的特点包括：
- 它在2024年8月成为了PlayStation 5上北美（美国和加拿大）、欧洲以及日本下载量最高的游戏；
- 发行后两周内，总销量已超过1800万份，总销售额约达人民币61. 65亿元；
- 实体豪华版和实体收藏版都供不应求；
- 在WeGame平台发行时经过了审查，对部分内容进行了修改。这种修改可能为了回应家长对孩子沉迷游戏以及其他潜在风险的担忧。

然而，关于游戏内容本身的具体特点，并未在提供的信息中提及。
